In [1]:
'''Loading libraries'''
import pandas as pd 
import numpy as np 
from scipy.stats import dirichlet, beta, nbinom, norm
from scipy.special import loggamma,gamma
import time
import matplotlib.pyplot as plt


'''
Notes: 
- lm(theta) is inf, but don't make sense because average theta is 7.42. 
- invest nan part
- when i add more columns the aceptance rate goes to 0. 
- Fix the saving files part, memory problms 
- Track running time
- create a function to save files/images with a labeled name 
'''


'''Important parameters I need to constantly change'''
k = 100

sim = 500
start_time = time.time()


In [2]:
'''Loading dataset'''
#filename = "C:\\Users\\raoki\\Documents\\GitHub\\project_spring2019\\Data\\data_final.csv"
filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final.csv"
#filename = "C:\\Users\\raque\\Google Drive\\SFU\\Project 2 - Spring 2019\\Data\\data_final_sub.csv"
data = pd.read_csv(filename, sep=',')


In [3]:
'''Saving time in the first part'''

data = data.iloc[:, 0:1000]
#data = data.sample(n=j).reset_index(drop=True)
data.shape


(2854, 1000)

In [4]:
'''Organizing columns names'''
lr = data.columns[[2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]
y = data.columns[3]
remove = data.columns[[0,1]]
#data_complete = data.copy()
for i in np.arange(19,data.shape[1]):
    data.iloc[:,i] = np.log(data.iloc[:,i]+1)

In [5]:
'''
Class to work with model parameters
I thought about using the default values as chain starting values, 
however, i encouter problems to change the size of arrays and matrices 
according with my currently k
'''
class parameters:
    def __init__(self, latent_v,latent_cj,latent_sk,latent_ev,latent_phi ,latent_tht, prediction):
        self.ln = latent_v #array with parameters that are only one number [0-c0,1-gamma0]
        self.la_cj = latent_cj #aaray J
        self.la_sk = latent_sk #array K
        self.la_ev = latent_ev #array V
        self.lm_phi = latent_phi #matrix (jk)
        self.lm_tht = latent_tht #matrix  (kv)      
        self.p = prediction #array [intercept, gender, 15 cancer types, k genes]


In [6]:
'''
Proposal distribution
NEED TO UPDATE

'''

#Proposal values for the parameters related to the factor model 
#Repete the parameters related to prediction and only propose
#new values for the factor analysys part
#output is the parameters class 
def proposal_f(current):
    new = parameters(np.random.normal(current.ln,0.05), 
                     np.random.normal(current.la_cj,2),
                     np.random.normal(current.la_sk,2),
                     np.random.normal(current.la_ev,0.05),
                     np.random.normal(current.lm_phi,0.000005), #remmeber that lm_phi sum up 1 in the line (genes)
                     np.random.normal(current.lm_tht,50), #remember the average value is 1600
                     current.p)
    #phi and tht can't be negative 
    new.lm_phi[new.lm_phi<0] = 0.0000001 #this number needs to be smaller 
    col_sums = new.lm_phi.sum(axis=0)
    new.lm_phi = new.lm_phi / col_sums[np.newaxis,:]
    new.lm_tht[new.lm_tht<0]=0
    return new

#Proposal values for the parameters related to logistic regression 
#Repete the parameters related to factor analysis part and propose
#new values for the logistc regression parameters 
#output is the parameters class 
def proposal_p(current):
    new = parameters(current.ln,current.la_cj ,current.la_sk, #current.la_pj, 
                     current.la_ev, current.lm_phi, current.lm_tht, 
                     np.random.normal(current.p,0.05))
    return new

In [14]:
def ratio_p(p_new,p_cur, data_P,k,y):
    sigma0 = 5
    sigma = 2
    mu0 = -len(p_new.p)
    mu = 1
    #H: beta~normal(mu,sigma2)
    H0 = (1/(sigma0*sigma0))*((p_cur.p[0]-mu0)*(p_cur.p[0]-mu0)-(p_new.p[0]-mu0)*(p_new.p[0]-mu0))*0.5
    H1 = (np.multiply((p_cur.p-mu),(p_cur.p-mu))-np.multiply((p_new.p-mu),(p_new.p-mu))).sum()
    H1 = H1 - (p_cur.p[0]-mu)*(p_cur.p[0]-mu)+(p_new.p[0]-mu)*(p_new.p[0]-mu)
    H1 = (H1*(len(p_new.p)-1)/sigma)*0.5
    
    #J: y~Log(xbeta)
    data_P = data_P.as_matrix()
    #print('dataP inside ratio',data_P[0:5])
    #data_P = np.append(np.array(np.repeat(1,data_P.shape[0])), data_P, axis=1)
    data_P = np.hstack((np.array(np.repeat(1,data_P.shape[0])).reshape(data_P.shape[0],1),data_P))
    data_P = np.hstack((data_P,np.transpose(p_cur.lm_tht)))
    #print('\n lm_tht',np.transpose(p_cur.lm_tht))
    xw_new = np.dot(data_P,p_new.p)
    xw_cur = np.dot(data_P,p_cur.p)
    J = (-np.log(1+np.exp(xw_new))+
          np.dot(y,xw_new)).sum()/((-np.log(1+np.exp(xw_cur))+
                                                        np.dot(y,xw_cur))).sum()
    #print('ratio - P',"%0.2f" % H0,"%0.2f" % H1,"%0.2f" % J)
    return (H0+H1+J)

In [8]:
'''
Creatint the MCMC for the model
MCMC(
startvalue = initial value for the parameters
iterations = 
data = complete data with all columns 
k = number of latent variables
remove, lr, y = columns names to be removed, presente only in the logistic regression part and y
)
'''
def MCMC(startvalue, iterations, data,k, lr,y):
    #print('all',data.head())
    '''Splitting dataset'''
    data_P = data[lr]
    #print('\n P',data_P.head())
    data_F = data.drop(lr,axis = 1)
    data_F = data_F.drop(y,axis = 1)
    y = data[y]
    a_P = 0
    a_F = 0
    '''
    Initialization of the chains
    Note: chain_f has elements from chain_p and vice-versa. Take care to not use incorrectly
    '''     
    chain_f = []
    chain_p = []
    chain_f.append(startvalue)
    chain_p.append(startvalue)
     
    for i in np.arange(1,iterations):
        '''Factor Analysis - Latent Features'''
        #use chain_f or chain_p don't make difference here because
        #the only parameters changed are the logistic regression and 
        #they aren't used in the factor analysis part. 
        param_new_f = proposal_f(chain_f[i-1])
        param_cur_f = chain_f[i-1] 
        if i%10 == 0: 
            a = a_F*10/i
            b = a_P*10/i
            print('iteration ',i,' acceptance ', "%0.2f" % a,'-', "%0.2f" % b)
        #prob_f = np.exp(posterior(param_new_f,data_F,data_P,y,k)-posterior(param_cur_f,data_F,data_P,y,k))
        prob_f = np.exp(ration_f(param_new_f,param_cur_f, data_F,k))
        if np.random.uniform(0,1,1)<prob_f:
            chain_f.append(param_new_f)
            a_F+=1
        else:
            chain_f.append(param_cur_f) 
        '''Logistic Regression - Prediction'''
        #chain_f[i] has the most update latent parameters and haven't changed the 
        #prediction parameters from [i-1] iteration
        param_new_p = proposal_p(chain_f[i-1])
        param_cur_p = chain_f[i-1]
        #print(param_new_p.p[0:20],param_cur_p.p[0:20])
        prob_p = np.exp(ratio_p(param_new_p,param_cur_p,data_P,k,y))
        if np.random.uniform(0,1,1)<prob_p:
            chain_p.append(param_new_p)
            a_P+=1
        else:
            chain_p.append(param_cur_p)     

    return chain_p, chain_f, a_P, a_F
        
        

In [9]:
'''Non informative prioris: dirichlet has only 1, gamma distribution with 1 average, etc'''
#UPDATE NUMBERS ACCORDING WITH POISSON AND LOG(N)
aux = len(lr)+1
data = data.drop(remove,axis = 1)
v = (data.shape[1]-aux)
j = data.shape[0]
start = parameters([3,6],#,5,41], #ln 
                   np.repeat(41,j), #la_cj
                   np.repeat(15,k), #la_sk
                   np.repeat(1,v), #la_ev
                   np.repeat(1/(data.shape[1]-aux),(data.shape[1]-aux)*k).reshape((data.shape[1]-aux),k),#lm_phi v x k 
                   np.repeat(7.42,(data.shape[0])*k).reshape(k,(data.shape[0])), #lm_theta k x j
                   np.concatenate(([-(k*7.42)], np.repeat(1,k+aux-1))))  #p, k+aux-1  because intercept is already counted



In [16]:
'''Ratio functions'''
def ration_f(p_new,p_cur, data_F,k):
    '''Priori Ration'''
    #log(1680)=7.42
    #J is samples and V is genes
    j = data_F.shape[0]
    v = data_F.shape[1]
    #A: phi_jk~Dir(eta_j)
    #print(loggamma(np.exp(np.sum(np.log(p_cur.la_ev)))),
    #       loggamma(np.exp(np.sum(np.log(p_new.la_ev)))))
    A0 = k*(loggamma(np.exp(np.sum(np.log(p_cur.la_ev))))-
           loggamma(np.exp(np.sum(np.log(p_new.la_ev)))))
    A1 = k*(np.sum(np.log(gamma(p_new.la_ev)))-np.sum(np.log(gamma(p_cur.la_ev))))
    A2 = np.matmul((p_new.la_ev-1),np.log(p_new.lm_phi)).sum()-np.matmul((p_cur.la_ev-1),np.log(p_cur.lm_phi)).sum()
    #print('A', p_cur.la_ev[0:5],np.log(p_cur.la_ev)[0:5],np.sum(np.log(p_cur.la_ev)))
    #B: eta_j~Gamma(a0,b0)
    a0 = 1/(2*v)
    b0 = 1/(2*v)
    B = (a0-1)*(np.log(p_new.la_ev)-np.log(p_cur.la_ev)).sum()+(p_cur.la_ev-p_new.la_ev).sum()/b0
    
    #C: theta_kl~Gamma(sk,cj)
    C0 = j*(loggamma(p_cur.la_sk).sum()-loggamma(p_new.la_sk).sum())+(
    p_cur.la_sk.sum()*np.log(p_cur.la_cj).sum()-p_new.la_sk.sum()*np.log(p_new.la_cj).sum())
    C1 = np.matmul(p_new.la_sk-1,np.log(p_new.lm_tht).sum(axis=1))-np.matmul(
        p_cur.la_sk-1,np.log(p_cur.lm_tht).sum(axis=1))
    C2 = np.divide(p_cur.lm_tht.sum(axis=0),p_cur.la_cj).sum()-np.divide(p_new.lm_tht.sum(axis=0),p_new.la_cj).sum()
    
    #D: sk~Gamma(gamma0,c0), gamma0 = c0 = (v*averageExpression)^0.5
    average4 = np.sqrt(np.sqrt(v*7.42))
    gamma0 = average4
    c0 = average4
    D = (gamma0-1)*(np.log(p_new.la_sk)-np.log(p_cur.la_sk)).sum()+(p_cur.la_sk-p_new.la_sk).sum()/c0
    
    #E: Cj~Gamma(a1,b1)
    a1 = average4
    b1 = average4
    E = (a1-1)*(np.log(p_new.la_cj)-np.log(p_cur.la_cj)).sum()+(p_cur.la_cj-p_new.la_cj).sum()/b1
    
    #F: gamma0~Gamma(a2,b2)
    average8 = np.sqrt(average4)
    a2 = average8
    b2 = average8
    F = (a2-1)*(np.log(p_new.ln[1])-np.log(p_cur.ln[1]))+(p_cur.ln[1]-p_new.ln[1])/b2
    
    #G: c0~Gamma(a3,b3)
    a3 = average8
    b3 = average8
    G = (a3-1)*(np.log(p_new.ln[0])-np.log(p_cur.ln[0]))+(p_cur.ln[0]-p_new.ln[0])/b3
    '''Likelihood'''
    #I: n_vj~Poisson(phi_vk theta_kj)
    I0 = np.transpose(np.log(np.matmul(p_new.lm_phi,p_new.lm_tht))-np.log(np.matmul(p_cur.lm_phi,p_cur.lm_tht)))
    #print(I0.shape,I0[0:5],data_F.head())
    I1 = np.multiply(data_F.as_matrix(),I0).sum()
    I2 = (np.matmul(p_cur.lm_phi,p_cur.lm_tht)-np.matmul(p_new.lm_phi,p_new.lm_tht)).sum()
    print('ratio - F',"%0.2f" % A0,"%0.2f" % A1,"%0.2f" % A2,"%0.2f" % B,"%0.2f" % C0,'inf',
          "%0.2f" % C1,"%0.2f" % C2,"%0.2f" % D,"%0.2f" % E, "%0.2f" % F,"%0.2f" % G,
         "%0.2f" % I1,"%0.2f" % I2,'end')
    return (A0+A1+A2+B+C0+C1+C2+D+E+F+G+I1+I2)

In [17]:
#data = data.drop(remove,axis=1)
start_time = time.time()
output_p, output_f, acept_P,acept_F = MCMC(start,sim,data,k,lr,y)
end_time = time.time() - start_time

print("--- %s seconds ---" % (time.time() - start_time))


#Don't make sense, some operations must be wrong

C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:53: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\raque\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


ratio - F 3.12 127.10 -881.87 -2511.79 205787.14 inf -inf -115094.66 -14.62 -39.40 -0.00 0.01 17469747.47 -4695276.70 end
ratio - F -249.91 280.09 887.29 2529.78 -94409.83 inf -inf -114277.43 -5.59 -17.19 0.00 0.00 17430355.50 -4675757.59 end
ratio - F -231.97 269.01 774.96 2209.98 -164630.21 inf -inf -114759.44 -4.21 -17.01 0.00 -0.05 17478277.60 -4695075.77 end
ratio - F -455.00 393.95 2289.16 6524.43 -425305.47 inf -inf -114933.48 1.17 -48.08 -0.00 0.02 17451312.69 -4683282.19 end
ratio - F -126.74 221.38 21.17 61.25 400471.84 inf -inf -114695.16 -18.15 -29.13 -0.00 -0.00 17451859.89 -4685212.80 end
ratio - F -162.39 224.14 360.79 1029.26 273290.37 inf -inf -114495.42 -15.38 -22.28 0.00 -0.01 17446103.07 -4681084.98 end
ratio - F -284.06 310.13 1042.63 2972.11 224055.86 inf -inf -114934.43 -13.87 -37.44 -0.00 -0.02 17461038.59 -4690739.89 end
ratio - F -313.16 317.83 1294.67 3690.74 -87865.76 inf -inf -115586.43 -5.34 -30.80 -0.00 0.01 17539444.91 -4721617.74 end
ratio - F -83.57 18

ratio - F -92.67 197.90 -173.02 -491.86 -256480.40 inf -inf -115183.06 -1.91 -20.46 0.00 0.01 17520777.44 -4711473.32 end
ratio - F -182.91 47.25 -1783.19 -5080.60 531217.08 inf -inf -114620.92 -22.88 -23.63 0.00 -0.04 17455107.16 -4686187.70 end
iteration  70  acceptance  0.00 - 0.43
ratio - F -119.39 212.26 3.92 12.55 130983.27 inf -inf -114828.13 -10.48 -3.26 -0.00 -0.00 17504603.57 -4706513.52 end
ratio - F -178.83 245.96 384.50 1096.96 -133624.86 inf -inf -115109.70 -7.49 -27.60 0.00 0.00 17484763.89 -4702930.85 end
ratio - F -209.42 256.74 623.12 1777.04 -383031.30 inf -inf -114358.75 -0.71 -24.17 -0.00 -0.02 17430694.78 -4674278.46 end
ratio - F -50.79 166.11 -407.25 -1159.32 -493999.18 inf -inf -114661.51 5.39 -26.63 0.00 0.00 17451920.81 -4685910.81 end
ratio - F -31.09 81.66 -1513.98 -4313.03 457580.88 inf -inf -114919.76 -22.60 -33.36 0.00 0.00 17469508.86 -4693271.45 end
ratio - F -281.74 293.93 1123.86 3203.86 441986.46 inf -inf -114769.69 -19.11 -23.88 0.00 -0.00 17479092

ratio - F -159.10 222.16 340.67 971.66 113066.97 inf -inf -115923.63 -10.09 -40.16 0.00 -0.02 17549084.58 -4728763.02 end
ratio - F 12.14 115.29 -1154.42 -3288.13 -71194.09 inf -inf -115200.96 -6.00 -31.12 0.00 0.01 17499862.88 -4705040.51 end
ratio - F -17.56 145.89 -674.07 -1919.51 12506.76 inf -inf -115258.32 -7.51 -25.48 -0.00 -0.01 17505441.03 -4711446.93 end
ratio - F 11.74 106.78 -1156.46 -3294.57 -125279.60 inf -inf -114730.92 -4.58 -28.67 -0.00 0.01 17459374.39 -4687498.31 end
ratio - F -144.86 220.77 205.56 586.86 342944.71 inf -inf -116359.12 -16.93 -33.72 -0.00 0.00 17605158.09 -4751380.36 end
ratio - F -44.30 166.91 -486.18 -1383.86 -90229.74 inf -inf -114568.98 -4.44 -22.13 0.00 0.01 17454131.04 -4685039.84 end
iteration  140  acceptance  0.00 - 0.43
ratio - F -150.03 215.67 292.79 835.39 76423.08 inf -inf -115123.89 -12.19 -31.11 -0.00 0.03 17488552.59 -4701862.55 end
ratio - F -149.49 221.19 249.78 713.23 -16940.24 inf -inf -114883.38 -8.95 -31.79 -0.00 0.01 17470562.36

ratio - F -103.13 195.82 -53.06 -149.75 -292500.42 inf -inf -115462.84 -0.26 -28.87 0.00 -0.02 17518626.12 -4715754.28 end
iteration  200  acceptance  0.00 - 0.40
ratio - F -240.01 281.92 772.83 2203.82 242838.06 inf -inf -114364.52 -14.27 -19.33 -0.00 -0.01 17440631.16 -4677518.50 end
ratio - F -73.13 175.24 -227.44 -646.89 27731.12 inf -inf -114644.40 -9.27 -25.15 -0.00 -0.00 17453188.54 -4686053.37 end
ratio - F -369.40 349.75 1675.22 4774.98 362495.73 inf -inf -114406.59 -15.97 -17.73 -0.00 -0.00 17438966.33 -4681443.73 end
ratio - F -66.69 181.51 -335.21 -953.82 -63607.08 inf -inf -115226.61 -6.54 -38.45 0.00 0.01 17489876.94 -4700985.71 end
ratio - F -118.32 49.15 -1643.32 -4681.48 502748.22 inf -inf -115351.33 -19.83 -44.89 0.00 0.00 17491902.07 -4703031.03 end
ratio - F -263.76 290.86 958.63 2732.92 289544.93 inf -inf -114718.42 -16.72 -39.26 -0.00 -0.01 17427805.41 -4678107.02 end
ratio - F -74.88 175.66 -211.74 -602.20 378082.70 inf -inf -114983.28 -16.49 -34.15 0.00 0.01 174

ratio - F 11.14 99.59 -1147.61 -3268.78 -642483.21 inf -inf -114893.68 7.64 -28.28 -0.00 -0.01 17468647.56 -4694229.46 end
ratio - F -693.20 25.35 -2256.29 -6428.54 126856.76 inf -inf -114562.66 -13.35 -35.74 -0.00 -0.00 17428584.08 -4673963.06 end
ratio - F -137.45 213.74 177.31 506.67 431998.92 inf -inf -115555.41 -19.80 -19.96 0.00 -0.00 17544468.84 -4726925.45 end
ratio - F -56.70 165.78 -341.51 -971.71 -482320.95 inf -inf -114685.30 5.26 -18.49 -0.00 0.01 17467913.13 -4690401.65 end
ratio - F -328.24 315.48 1466.09 4179.29 -593661.68 inf -inf -114941.12 5.88 -34.51 0.00 -0.04 17470246.56 -4691369.05 end
iteration  270  acceptance  0.00 - 0.41
ratio - F -288.36 289.43 1220.03 3477.89 -333410.33 inf -inf -114908.11 0.34 -35.07 0.00 -0.02 17465534.84 -4691258.34 end
ratio - F -99.80 206.15 -152.50 -433.38 -517893.01 inf -inf -114361.86 5.07 -35.32 -0.00 0.02 17409477.83 -4666738.08 end
ratio - F -112.60 194.27 53.83 154.38 -163649.83 inf -inf -115333.66 -3.88 -40.95 -0.00 0.03 175005

ratio - F 11.32 111.20 -1022.87 -2914.13 261461.77 inf -inf -114448.49 -15.47 -30.63 -0.00 0.00 17424866.10 -4673204.82 end
ratio - F 11.90 107.44 -1148.59 -3271.70 249069.39 inf -inf -115003.87 -14.65 -42.93 -0.00 0.00 17453605.06 -4688994.14 end
ratio - F -173.91 236.88 392.35 1119.63 -750446.26 inf -inf -115083.75 10.55 -43.31 -0.00 0.01 17473515.53 -4691392.82 end
ratio - F -163.46 228.99 339.16 967.76 -708574.71 inf -inf -114940.09 7.83 -30.82 0.00 0.00 17478822.87 -4694497.04 end
ratio - F 12.09 107.22 -1121.80 -3195.64 524642.84 inf -inf -115079.07 -19.59 -30.30 -0.00 -0.02 17493119.24 -4699682.56 end
ratio - F -294.71 302.20 1204.35 3433.10 -407529.32 inf -inf -115226.13 0.96 -23.36 0.00 -0.02 17506393.52 -4710333.81 end
ratio - F -194.51 254.20 487.21 1389.86 52573.61 inf -inf -114482.15 -9.95 -16.10 -0.00 -0.00 17451895.82 -4685407.38 end
ratio - F -2466.48 -30.53 -2661.91 -7584.32 -97289.66 inf -inf -115093.73 -7.56 -26.54 -0.00 0.00 17486609.32 -4702158.02 end
ratio - F 12.

ratio - F -83.13 183.19 -174.59 -496.29 210328.63 inf -inf -115370.98 -16.18 -45.93 0.00 -0.01 17499858.75 -4703620.41 end
ratio - F -99.33 198.98 -110.45 -313.51 6850.49 inf -inf -115271.60 -9.66 -27.45 -0.00 -0.04 17515469.10 -4710332.38 end
ratio - F -105.52 200.74 -62.39 -176.52 299674.25 inf -inf -115042.26 -15.72 -19.91 0.00 -0.01 17494305.11 -4704269.52 end
iteration  400  acceptance  0.00 - 0.35
ratio - F -94.52 77.16 -1751.43 -4989.70 94419.79 inf -inf -114056.90 -10.58 -27.72 -0.00 -0.02 17394463.73 -4659703.10 end
ratio - F -250.58 286.94 848.61 2419.70 -117463.06 inf -inf -114617.20 -5.31 -15.95 0.00 0.01 17468031.39 -4691835.31 end
ratio - F -14.83 138.36 -662.35 -1885.93 -376586.88 inf -inf -114228.66 0.37 -26.56 0.00 0.01 17415632.06 -4668772.04 end
ratio - F -114.48 208.81 -22.95 -64.04 -670067.23 inf -inf -114699.12 8.52 -18.93 0.00 0.00 17457815.80 -4690890.99 end
ratio - F -180.75 227.62 522.65 1490.44 376270.06 inf -inf -115208.81 -19.87 -1.58 0.00 0.01 17546541.37 

ratio - F -9.90 143.97 -767.80 -2186.58 -19769.69 inf -inf -115043.08 -9.66 -37.80 0.00 0.03 17469522.45 -4693786.36 end
ratio - F -44.79 154.89 -401.99 -1144.43 428739.04 inf -inf -114590.56 -17.38 -4.48 0.00 -0.01 17478267.38 -4695935.61 end
ratio - F -134.27 205.93 196.14 560.19 352228.70 inf -inf -114418.03 -17.18 -23.42 0.00 -0.02 17438172.67 -4677971.22 end
ratio - F -392.37 363.12 1829.05 5213.30 -367617.34 inf -inf -116170.71 1.50 -31.30 -0.00 -0.00 17587305.19 -4745059.90 end
ratio - F -273.77 290.18 1066.20 3039.74 275044.48 inf -inf -114903.39 -15.97 -23.70 -0.00 0.01 17481417.06 -4699194.73 end
ratio - F -94.91 193.52 -120.90 -343.28 198390.14 inf -inf -114702.52 -14.25 -39.38 -0.00 0.00 17438277.59 -4678291.17 end
ratio - F -154.30 221.27 296.88 847.54 550305.21 inf -inf -115815.92 -23.04 -37.24 -0.00 -0.00 17547653.49 -4726879.02 end
iteration  470  acceptance  0.00 - 0.38
ratio - F -98.14 187.44 -49.59 -140.20 230713.51 inf -inf -114429.75 -12.51 -26.62 -0.00 0.03 174303

In [ ]:
np.exp(8)/2800

In [ ]:
'''Organizing outputs'''

#TRANSFORM IN A FUNCTION TO STOP HAVE MEMORY PROBLEMS 

#Logistic Regression Parameters 
output_logistic = np.concatenate((output_p[0].p,output_p[1].p),axis = 0)
#Factor Model parameters
output_factor_ln = np.concatenate((output_f[0].ln,output_f[1].ln),axis = 0)
output_factor_la_sk = np.concatenate((output_f[0].la_sk,output_f[1].la_sk),axis = 0)
#Matrix AxB is saved as a line A*B elements 
output_factor_lm_phi= np.concatenate((output_f[0].lm_phi,output_f[1].lm_phi),axis = 0)
output_factor_lm_tht= np.concatenate((output_f[0].lm_tht,output_f[1].lm_tht),axis = 0)


for i in np.arange(2,sim):
    output_logistic = np.concatenate((output_logistic,output_p[i].p),axis = 0)
    output_factor_ln = np.concatenate((output_factor_ln,output_f[1].ln),axis = 0)
    output_factor_la_sk = np.concatenate((output_factor_la_sk,output_f[1].la_sk),axis = 0)

output_logistic = output_logistic.reshape(sim,len(output_p[0].p) )    
output_factor_ln = output_factor_ln.reshape(sim,len(output_f[0].ln) )
output_factor_la_sk = output_factor_la_sk.reshape(sim,len(output_f[0].la_sk) )   

np.savetxt('Data\\output_logistic.txt', output_logistic, delimiter=',')  
np.savetxt('Data\\output_factor_ln.txt', output_factor_ln, delimiter=',')  
np.savetxt('Data\\output_factor_la_sk.txt', output_factor_la_sk, delimiter=',')  

'''
    output_factor_lm_phi = np.concatenate((output_factor_lm_phi,output_f[1].lm_phi),axis = 0)
    output_factor_lm_tht = np.concatenate((output_factor_lm_tht,output_f[1].lm_tht),axis = 0)
    
output_factor_lm_phi = output_factor_lm_phi.reshape(sim,output_f[0].lm_phi.shape[0]*output_f[0].lm_phi.shape[1])   
output_factor_lm_tht = output_factor_lm_tht.reshape(sim,output_f[0].lm_tht.shape[0]*output_f[0].lm_tht.shape[1])

np.savetxt('Data\\output_factor_lm_phi.txt', output_factor_lm_phi, delimiter=',')  
np.savetxt('Data\\output_factor_lm_tht.txt', output_factor_lm_tht, delimiter=',')  
'''

In [ ]:
#acceptance and rejection  problem 
#create a easy way to identify paramerters based on name
#test several parameters at once

In [ ]:
plt.plot(np.arange(0,len(output_logistic[:,0])),output_logistic[:,0], 'r-', alpha=1)
plt.xlabel('Logistic Regression - intercept')
plt.savefig('Data\\lr_intercept.png')

plt.plot(np.arange(0,len(output_logistic[:,1])),output_logistic[:,1], 'r-', alpha=1)
plt.xlabel('Logistic Regression - gender')
plt.savefig('Data\\lr_gender.png')

plt.plot(np.arange(0,len(output_logistic[:,10])),output_logistic[:,10], 'r-', alpha=1)
plt.xlabel('Logistic Regression - cancer type (one of them)')
plt.savefig('Data\\lr_cancertype.png')

plt.plot(np.arange(0,len(output_logistic[:,30])),output_logistic[:,30], 'r-', alpha=1)
plt.xlabel('Logistic Regression - k (one of them)')
plt.savefig('Data\\lr_k.png')

plt.plot(np.arange(0,len(output_factor_ln[:,2])),output_factor_ln[:,2], 'r-', alpha=1)
plt.xlabel('Factor Analysis - parameter')
plt.savefig('Data\\fa_1.png')

